In [1]:
!pip install --quiet -U langchain==0.2.16
!pip install --quiet -U langchain_openai==0.1.23
!pip install --quiet -U langgraph==0.2.19
!pip install --quiet -U langchainhub==0.1.21
!pip install --quiet -U tavily-python==0.4.0
!pip install --quiet -U langchain-community==0.2.16
!pip install --quiet -U python-dotenv==1.0.1


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [1]:
from dotenv import load_dotenv
import os

load_dotenv("config.env")
assert os.getenv("OPENAI_API_KEY") is not None, "Missing OpenAI API key!"
assert os.getenv("TAVILY_API_KEY") is not None, "Missing Tavily API key!"




In [2]:
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.prompts import PromptTemplate
from openai import OpenAI

In [3]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Optional

class HbotState(TypedDict, total=False):
    topic: Optional[str]
    summary: Optional[str]
    search_results: Optional[str]
    patient_answer: Optional[str]
    quiz_question: Optional[str]
    patient_answer: Optional[str]
    grading_result: Optional[str]



In [4]:
def ask_topic(state: HbotState):
    # if topic already exists
    topic = state.get("topic")

    if not topic:
        topic = input("Enter a health topic: ").strip()
        if not topic:
            print("No input.")
            return {"end": True}

    print(f"Starting with topic: {topic}")
    return {"topic": topic}


In [5]:
def tavily_search(state: HbotState):
    from langchain_community.tools.tavily_search import TavilySearchResults
    tavily_tool = TavilySearchResults(max_results=5)

    print(f"Searching regarding: {state['topic']}...\n")
    search = tavily_tool.invoke({"query": state['topic']})
    search_results = "\n".join([doc["content"] for doc in search])
    return {"search_results": search_results}



In [6]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7 , base_url="https://openai.vocareum.com/v1")
def result_summary(state: HbotState):
    prompt = PromptTemplate.from_template("""
    Explain following medical condition,
    using,patient-friendly language and key points take care during that condition.the key points should be in below format:
    <<**IMPORTANT**>>
    => 
    =>
    =>
    Information:
    {search_results}
    """)

    response = llm.invoke(prompt.format(search_results=state["search_results"]))
    summary = response.content
    print("\n Here is the info:\n")
    print(summary)
    return {"summary": summary}



In [7]:
def wait_for_quiz(state: HbotState):
    input("\n Lets begin a quick QUIZZ...")
    return {"summary": state["summary"]}


In [8]:
def ask_quiz(state: HbotState):

    prompt = PromptTemplate.from_template("""
    Based ONLY on the summary, create ONE multiple-choice question 
    with four options.Do NOT include the correct answer in the output.
    Summary:
    {summary}
    """)

    quiz = llm.invoke(prompt.format(summary=state["summary"]))
    quiz_question = quiz.content
    print("\n Quiz Question:\n")
    print(quiz_question)
    return {"quiz_question": quiz_question}



In [9]:
def get_answer(state: HbotState):
    answer = input("\n Enter your answer: ").strip().upper()
    return {"patient_answer": answer}



In [10]:
def Quiz_rank(state: HbotState):
    prompt = PromptTemplate.from_template("""
    Grade the patient's answer to the question below.
    Quiz Question:
    {quiz_question}
    Patient's Answer: {patient_answer}
    Use ONLY the summary below. Provide:
    - Letter grade (A–c)
    - Short, explanation what he should do while he is going through the condition.
    Summary:
    {summary}
    """)

    result = llm.invoke(prompt.format(
        quiz_question=state["quiz_question"],
        patient_answer=state["patient_answer"],
        summary=state["summary"]
    ))

    grading_result = result.content
    print("\n Here's your result:\n")
    print(grading_result)
    return {"grading_result": grading_result}



In [11]:
def ask_to_continue(state: HbotState):
    choice = input("\nWould you like togo further? (yes/no): ").strip().lower()

    if choice in ["yes", "y"]:
        new_topic = input(" Enter a new health topic you'd like to learn about: ").strip()
        if not new_topic:
            print(" No topic entered.")
            return {"next": END}
        print("\n starting a new topic.\n")
        return {"topic": new_topic, "next": "restart"}
    else:
        print("\n Thank you!!")
        # return {"next": END}
        return {"topic": None, "next": END}


In [12]:
graph = StateGraph(HbotState)
graph.add_node("ask_topic", ask_topic)
graph.add_node("tavily_search", tavily_search)
graph.add_node("result_summary", result_summary)
graph.add_node("wait_for_quiz", wait_for_quiz)
graph.add_node("ask_quiz", ask_quiz)
graph.add_node("get_answer", get_answer)
graph.add_node("Quiz_rank", Quiz_rank)
graph.add_node("ask_to_continue", ask_to_continue)

# order

graph.set_entry_point("ask_topic")
graph.add_edge("ask_topic", "tavily_search")
graph.add_edge("tavily_search", "result_summary")
graph.add_edge("result_summary", "wait_for_quiz")
graph.add_edge("wait_for_quiz", "ask_quiz")
graph.add_edge("ask_quiz", "get_answer")
graph.add_edge("get_answer", "Quiz_rank")
graph.add_edge("Quiz_rank", "ask_to_continue")

# Restart or end

graph.add_conditional_edges(
    "ask_to_continue",
    lambda x: x["next"],
    {"restart": "ask_topic", END: END}
)
app = graph.compile()



In [ ]:

user_topic = input("Enter a health topic you'd like to learn about: ")
result = app.invoke({"topic": user_topic})


Enter a health topic you'd like to learn about: cough
Starting with topic: cough
Searching regarding: cough...


 Here is the info:

### Understanding Coughing: A Patient-Friendly Explanation

A cough is your body's way of protecting itself. When something irritates your throat or airways—like dust, germs, or mucus—your body responds by coughing. This helps clear out those irritants, keeping your airways clean and healthy. While occasional coughing is normal and can help you clear your throat, a persistent cough may signal a more serious issue that needs attention. 

---

### <<**IMPORTANT**>>
=> **What Causes a Cough?**
   - Coughs can be triggered by many things, including allergies, infections (like colds and flu), chronic lung conditions (like asthma), and even acid reflux.
   - Sometimes, irritants like smoke, dust, or strong odors can also cause coughing.

=> **When to Seek Medical Help:**
   - If your cough lasts more than three weeks (acute cough) or longer than eight weeks (ch